In [80]:
import torch
import tensorflow as tf
import numpy as np
import os
from os import path
from collections import OrderedDict

In [81]:
model_size = "large"
tf_checkpoint = f'/home/shtoshni/Research/mandar_coref/coref/data/spanbert_{model_size}/model.max.ckpt'
output_dir = '/home/shtoshni/Research/litbank_coref/resources/mentions'
if not path.exists(output_dir):
    os.mkdir(output_dir)


In [82]:
model_vars = tf.train.list_variables(tf_checkpoint)

for name, shape in model_vars:
    if 'bert' in name or 'Adam' in name or 'beta' in name or 'global_step' in name:
        continue
    print(name, shape)

antecedent_distance_emb [10, 20]
coref_layer/antecedent_distance_emb [10, 20]
coref_layer/f/output_bias [3092]
coref_layer/f/output_weights [6184, 3092]
coref_layer/same_speaker_emb [2, 20]
coref_layer/segment_distance/segment_distance_embeddings [3, 20]
coref_layer/slow_antecedent_scores/hidden_bias_0 [3000]
coref_layer/slow_antecedent_scores/hidden_weights_0 [9356, 3000]
coref_layer/slow_antecedent_scores/output_bias [1]
coref_layer/slow_antecedent_scores/output_weights [3000, 1]
genre_embeddings [7, 20]
mention_scores/hidden_bias_0 [3000]
mention_scores/hidden_weights_0 [3092, 3000]
mention_scores/output_bias [1]
mention_scores/output_weights [3000, 1]
mention_word_attn/output_bias [1]
mention_word_attn/output_weights [1024, 1]
output_bias [1]
output_weights [20, 1]
span_width_embeddings [30, 20]
span_width_prior_embeddings [30, 20]
src_projection/output_bias [3092]
src_projection/output_weights [3092, 3092]
width_scores/hidden_bias_0 [3000]
width_scores/hidden_weights_0 [20, 3000]


## Mention vars

In [83]:
for name, shape in model_vars:
    if 'bert' in name or 'Adam' in name:
        continue
    if 'mention' in name or 'width' in name:
        print(name, shape)

mention_scores/hidden_bias_0 [3000]
mention_scores/hidden_weights_0 [3092, 3000]
mention_scores/output_bias [1]
mention_scores/output_weights [3000, 1]
mention_word_attn/output_bias [1]
mention_word_attn/output_weights [1024, 1]
span_width_embeddings [30, 20]
span_width_prior_embeddings [30, 20]
width_scores/hidden_bias_0 [3000]
width_scores/hidden_weights_0 [20, 3000]
width_scores/output_bias [1]
width_scores/output_weights [3000, 1]


In [84]:
for ment_emb in ['attn', 'endpoint']:
    pytorch_dump_path = path.join(output_dir, f'mention_ontonotes_{model_size}_{ment_emb}.pt')
    model_state_dict = OrderedDict()
    
    tf_var_to_torch_var = OrderedDict()

    tf_var_to_torch_var['mention_scores/hidden_weights_0'] = 'mention_mlp.fc_layers.0.weight'
    tf_var_to_torch_var['mention_scores/hidden_bias_0'] = 'mention_mlp.fc_layers.0.bias'
    tf_var_to_torch_var['mention_scores/output_weights'] = 'mention_mlp.fc_layers.1.weight'
    tf_var_to_torch_var['mention_scores/output_bias'] = 'mention_mlp.fc_layers.1.bias'
    tf_var_to_torch_var['mention_word_attn/output_weights'] = 'mention_attn.weight'
    tf_var_to_torch_var['mention_word_attn/output_bias'] = 'mention_attn.bias'

    tf_var_to_torch_var['width_scores/hidden_weights_0'] = 'span_width_mlp.fc_layers.0.weight'
    tf_var_to_torch_var['width_scores/hidden_bias_0'] = 'span_width_mlp.fc_layers.0.bias'
    tf_var_to_torch_var['width_scores/output_weights'] = 'span_width_mlp.fc_layers.1.weight'
    tf_var_to_torch_var['width_scores/output_bias'] = 'span_width_mlp.fc_layers.1.bias'

    tf_var_to_torch_var['span_width_embeddings'] = 'span_width_embeddings.weight'
    tf_var_to_torch_var['span_width_prior_embeddings'] = 'span_width_prior_embeddings.weight'

    for tf_var, torch_var in tf_var_to_torch_var.items():
        numpy_mat = torch.from_numpy(tf.train.load_variable(tf_checkpoint, tf_var))
        if ment_emb == 'endpoint' and tf_var == 'mention_scores/hidden_weights_0':
            if model_size == 'base':
                cutoff = 2 * 768 + 20
            else:
                cutoff = 2 * 1024 + 20
            print(tf_var, cutoff)
            numpy_mat = numpy_mat[:cutoff, :]
        if 'weights' in tf_var:
            numpy_mat = np.transpose(numpy_mat)
            print(torch_var, numpy_mat.shape)

        model_state_dict[torch_var] = numpy_mat
            

    torch.save(model_state_dict, pytorch_dump_path)

mention_mlp.fc_layers.0.weight torch.Size([3000, 3092])
mention_mlp.fc_layers.1.weight torch.Size([1, 3000])
mention_attn.weight torch.Size([1, 1024])
span_width_mlp.fc_layers.0.weight torch.Size([3000, 20])
span_width_mlp.fc_layers.1.weight torch.Size([1, 3000])
mention_scores/hidden_weights_0 2068
mention_mlp.fc_layers.0.weight torch.Size([3000, 2068])
mention_mlp.fc_layers.1.weight torch.Size([1, 3000])
mention_attn.weight torch.Size([1, 1024])
span_width_mlp.fc_layers.0.weight torch.Size([3000, 20])
span_width_mlp.fc_layers.1.weight torch.Size([1, 3000])
